### Exceptions

* list of builtin exceptions: https://docs.python.org/3/library/exceptions.html#exception-hierarchy

Exception handling started simple:

> Very early Python versions used simple strings to signalize errors. Later, Python allowed raising arbitrary classes, and added specialized exception classes to the standard library.

Now: Exceptions are classes and contain all information about the error: the type, value and traceback.

Typically, we can reuse many standard exceptions

In [2]:
try:
    1 / 0
except ZeroDivisionError as exc:
    pass

Notes:
    
* we do not need to catch a specific exception - but it (much) better style
* we can name multiple exceptions in a single clause
* we can have multiple except branches
* we can have an else branch

The `exc` value is scoped to the try statement

> This means the exception must be assigned to a different name to be able to refer to it after the except clause.

In [3]:
class B(Exception):
    pass

class C(B):
    pass

class D(C):
    pass

for cls in [B, C, D]:
    try:
        raise cls()
    except D:
        print("D")
    except C:
        print("C")
    except B:
        print("B")

B
C
D


Exceptions have an args attribute

In [5]:
try:
    raise Exception('spam', 'eggs')
except Exception as inst:
    print(type(inst))    # the exception instance
    print(inst.args)     # arguments stored in .args
    print(inst)          # __str__ allows args to be printed directly,
                         # but may be overridden in exception subclasses


<class 'Exception'>
('spam', 'eggs')
('spam', 'eggs')


### Exception classes

BaseException is the common base class of all exceptions. One of its subclasses, Exception, is the base class of all the non-fatal exceptions.

```
BaseException
 ├── BaseExceptionGroup
 ├── GeneratorExit
 ├── KeyboardInterrupt
 ├── SystemExit
 └── Exception
      ├── ArithmeticError
      │    ├── FloatingPointError
      │    ├── OverflowError
      │    └── ZeroDivisionError
      ...
```

### Exception chaining

* interpreter can propagate one exception at a time
* multiple exceptions may occur

In [8]:
def compute(a, b):
    try:
        a/b
    except Exception as exc:
        log(exc)

def log(exc):
    raise IOError("unavailble")

In [44]:
# compute(1, 0)

```
---------------------------------------------------------------------------
ZeroDivisionError                         Traceback (most recent call last)
Cell In[8], line 3, in compute(a, b)
      2 try:
----> 3     a/b
      4 except Exception as exc:

ZeroDivisionError: division by zero

During handling of the above exception, another exception occurred:

OSError                                   Traceback (most recent call last)
Cell In[9], line 1
----> 1 compute(1, 0)

Cell In[8], line 5, in compute(a, b)
      3     a/b
      4 except Exception as exc:
----> 5     log(exc)

Cell In[8], line 8, in log(exc)
      7 def log(exc):
----> 8     raise IOError("unavailble")

OSError: unavailble
```

In [19]:
try:
    compute(1, 0)
except Exception as exc:
    err = exc

In [21]:
err.args

('unavailble',)

In [25]:
err.__cause__

In [26]:
err.__context__

ZeroDivisionError('division by zero')

In [34]:
err.__traceback__

In [43]:
err.__traceback__.tb_frame.f_code.co_consts # finding 1 and 0 again

(1, 0, None)

In [36]:
err.__traceback__.tb_lasti

24

In [37]:
err.__traceback__.tb_lineno

2

In [38]:
err.__traceback__.tb_next

### What is cause, what is context?

> This PEP proposes three standard attributes on exception instances: the `__context__` attribute for implicitly chained exceptions, the `__cause__` attribute for explicitly chained exceptions, and the `__traceback__` attribute for the traceback. -- PEP 3134 – Exception Chaining and Embedded Tracebacks

* cause - explicit
* context - implicit

A cause can be added via `raise ... from`

In [50]:
# raise Exception("fail") from Exception("from")

Another example

In [51]:
def func():
    raise ConnectionError

try:
    func()
except ConnectionError as exc:
    raise RuntimeError('Failed to open database') from exc

RuntimeError: Failed to open database

### Reraising an exception

In [47]:
try:
    raise NameError('HiThere')
except NameError:
    print('An exception flew by!')
    # raise

An exception flew by!
